# Probabilistic Data Generator in Stan
This notebook contains the Stan code to learn the parameters for the probabilistic data generator from real data.

In [1]:
%matplotlib inline

import pystan
import pandas as pd
import numpy as np
import scipy.io as sio
import pickle

In [2]:
# Stan code for the model
model_code="""
data{
    int I;
    int R;
    int K;
    real eta;
    real zeta;
    int J;
    real x[I, J];
    vector[R] b[I];
    matrix[K,R] lambda;
}
parameters{
    matrix[K,R] omega;
    vector<lower=0>[K] phi_alpha;
    vector<lower=0>[K] phi_beta;
}
transformed parameters{
    simplex[K] theta[I];
    for (i in 1:I){
        theta[i] = softmax(omega*b[i]);
    }
}
model{
    for (k in 1:K){
        for (r in 1:R){
            omega[k,r] ~ normal(0, lambda[k,r]);    //prior
        }
    }
    for (k in 1:K){
        phi_alpha[k] ~ exponential(eta);    //prior
    }
    for (k in 1:K){
        phi_beta[k] ~ exponential(zeta);    //prior
    }
    for (i in 1:I){
        for (j in 1:J){
            x[i,j] ~ gamma(dot_product(phi_alpha, theta[i]),dot_product(phi_beta, theta[i]));
        }
    }
    for (i in 1:I){
        for (j in 1:J){
            real prob_state[K];
            for (k in 1:K){
                prob_state[k] = log(theta[i, k]);
            }
            increment_log_prob(log_sum_exp(prob_state));
        }
    }
}
"""

In [3]:
# Initialise Stan model
model = pystan.StanModel(model_code=model_code, verbose=True)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_f63f0f63d888d9d2bb963c008cef4695 NOW.
INFO:pystan:OS: linux, Python: 3.6.1 |Continuum Analytics, Inc.| (default, May 11 2017, 13:09:58) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)], Cython 0.25.2


Compiling /tmp/tmp6ub0ytb0/stanfit4anon_model_f63f0f63d888d9d2bb963c008cef4695_3317670944282290213.pyx because it changed.
[1/1] Cythonizing /tmp/tmp6ub0ytb0/stanfit4anon_model_f63f0f63d888d9d2bb963c008cef4695_3317670944282290213.pyx
building 'stanfit4anon_model_f63f0f63d888d9d2bb963c008cef4695_3317670944282290213' extension
creating /tmp/tmp6ub0ytb0/tmp
creating /tmp/tmp6ub0ytb0/tmp/tmp6ub0ytb0
gcc -pthread -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -DBOOST_RESULT_OF_USE_TR1 -DBOOST_NO_DECLTYPE -DBOOST_DISABLE_ASSERTS -I/tmp/tmp6ub0ytb0 -I/home/ayman/anaconda3/envs/hsbc/lib/python3.6/site-packages/pystan -I/home/ayman/anaconda3/envs/hsbc/lib/python3.6/site-packages/pystan/stan/src -I/home/ayman/anaconda3/envs/hsbc/lib/python3.6/site-packages/pystan/stan/lib/stan_math_2.15.0 -I/home/ayman/anaconda3/envs/hsbc/lib/python3.6/site-packages/pystan/stan/lib/stan_math_2.15.0/lib/eigen_3.2.9 -I/home/ayman/anaconda3/envs/hsbc/lib/python3.6/site-packages/pystan/stan/l

In [4]:
# Load data
with open('./model_data.pickle', 'rb') as f:
    model_data = pickle.load(f)

In [5]:
# Fit the model
fit = model.sampling(data=model_data, verbose=True)
fit

Inference for Stan model: anon_model_f63f0f63d888d9d2bb963c008cef4695.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

               mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
omega[0,0]    -0.37  3.7e-3   0.17   -0.7  -0.48  -0.37  -0.26  -0.05   1994    1.0
omega[1,0]     0.07  3.0e-3   0.15  -0.22  -0.03   0.07   0.17   0.38   2606    1.0
omega[0,1]     0.32  3.5e-3   0.18  -0.02    0.2   0.32   0.44   0.65   2459    1.0
omega[1,1]     -0.1  3.1e-3   0.17  -0.42  -0.21   -0.1   0.02   0.23   2936    1.0
omega[0,2]    -0.21  5.7e-3   0.31  -0.83  -0.42  -0.21-1.2e-3   0.43   3091    1.0
omega[1,2]     0.29  5.8e-3   0.32  -0.34   0.07   0.29    0.5   0.92   3037    1.0
phi_alpha[0]   3.28  4.7e-3   0.17   3.01   3.17   3.26   3.37   3.67   1279    1.0
phi_alpha[1]   2.01  4.5e-3   0.16   1.62   1.92   2.03   2.12   2.26   1275    1.0
phi_beta[0]  1.7e-6  9.5e-9 3.2e-7 8.8e-7 1.5e-6 1.7e-6

In [6]:
# Compare with real model parameters
with open('./init_dict.pickle', 'rb') as f:
    init_dict = pickle.load(f)

init_dict

{'omega': array([[-0.57378686,  0.16133765, -0.15473656],
        [-0.235538  , -0.17741711,  0.22316592]]),
 'phi_alpha': array([ 2079.92659863,  1183.7307828 ]),
 'phi_beta': array([ 787.59853677,  146.9759866 ]),
 'theta': array([[ 0.73051324,  0.26948676],
        [ 0.41642511,  0.58357489],
        [ 0.63123376,  0.36876624],
        [ 0.66969712,  0.33030288],
        [ 0.35642927,  0.64357073],
        [ 0.50141779,  0.49858221],
        [ 0.61518261,  0.38481739],
        [ 0.32894973,  0.67105027],
        [ 0.68451695,  0.31548305],
        [ 0.38438656,  0.61561344],
        [ 0.31355078,  0.68644922],
        [ 0.22122967,  0.77877033],
        [ 0.54364978,  0.45635022],
        [ 0.59818873,  0.40181127],
        [ 0.23609702,  0.76390298],
        [ 0.38294845,  0.61705155],
        [ 0.32551209,  0.67448791],
        [ 0.42751848,  0.57248152],
        [ 0.45680331,  0.54319669],
        [ 0.52661623,  0.47338377],
        [ 0.4735891 ,  0.5264109 ],
        [ 0.6284876